In [42]:

import numpy as np
import matplotlib.pyplot as plt
import os
import platform
import sys
import os 

sys.path.append("C:\\Program Files\\Lumerical\\v242\\api\\python") # this line is to direct the python script ot the lumerical python api 
import lumapi

current_directory= os.getcwd()
current_directory= r"{}".format(current_directory)
sys.path.append(current_directory)

focal_length = 100.0e-6
LensDiameter= 20.0e-6
SubstrateIndex= 1.46 
PillarIndex= 3.4757
period = 0.4e-6 
wavelength = 1.55e-6

unit_cell_library_file_name= 'EH_and_phase_vs_radius_interp_fdtd_si_sio2_sept13.mat'
zemax_phase_file_name=  'output_x_y_rad_apexPHAZE_optimized.txt'

In [82]:
fdtd= lumapi.FDTD()

setup_script= """
# create an aperture matching the diameter of the metalens 
R = getnamed("metalens","lens_radius");
W = getnamed("metalens::substrate","x span");
V = R*exp(1i*linspace(0,2*pi,200));
V = [W/2,0; W/2,-W/2; -W/2,-W/2; -W/2,W/2; W/2,W/2; W/2,0; real(V),imag(V)];
setnamed("aperture","vertices",V);
"""
fdtd.set("setup script",  setup_script)

radius_phase_file = current_directory+ '\\2_meta_library\\unit cell'+'\\' + unit_cell_library_file_name
zfile= current_directory+ '\\'+ zemax_phase_file_name

# #==================================================
# Add FDTD
# #==================================================
fdtd.addfdtd()




# create an aperture matching the diameter of the metalens 
# myscript= """
# R = getnamed("metalens","lens radius");
# W = getnamed("metalens::substrate","x span");
# V = R*exp(1i*linspace(0,2*pi,200));
# V = [W/2,0; W/2,-W/2; -W/2,-W/2; -W/2,W/2; W/2,W/2; W/2,0; real(V),imag(V)];
# setnamed("aperture","vertices",V);
# """
# fdtd.set("script",myscript)

# #==================================================
# Add the metalens geometry 
# #==================================================
fdtd.addstructuregroup()
fdtd.set("name", "metalens")
fdtd.adduserprop("radius_vs_phase_data_file", 1, radius_phase_file)
fdtd.adduserprop("zemax_optimized_phase_file", 1,zfile)
fdtd.adduserprop("focal_length", 2, focal_length)
fdtd.adduserprop("lens_radius", 2, LensDiameter/2)
fdtd.adduserprop("substrate_index", 0, SubstrateIndex)
fdtd.adduserprop("pillar_index", 0, PillarIndex)



myscript =f"""deleteall;
matlabload(radius_vs_phase_data_file);
?mat_sub;
np = round({LensDiameter/2}/ {period});
x_mask = period * (-np:1:np);
y_mask = period * (-np:1:np);

addrect; # substrate
set("name", "substrate");
set("index",substrate_index);

# ## set("material", "sio2");
# if (mat_sub=="<Object defined dielectric>") {{
#     set("index",substrate_index);
# }}

set("x span",3*max(x_mask));
set("y span",3*max(x_mask));
set("z max",0);
set("z min",-2*{wavelength});

M=readdata(zemax_optimized_phase_file);
len= length(M)/3;
for(i=1:len) {{
    addcircle({{"name":"pillar", "x":M(i),"y":M(len+i),"radius":M(2*len+i)}});
}}
select("pillar");
set("index",index_pillar);
# set("material", "si");


set("z min",0);
set("z max",1.3e-6);
?height;
"""

fdtd.set("script", myscript)

# #==================================================
# Add the aperture
# #==================================================

fdtd.add2dpoly()
fdtd.set("name", "aperture")
fdtd.set("z", -0.05e-6)
R = LensDiameter/2
W = 3* 10e-6
apertureP= np.linspace(0, 2*np.pi, 200)
V = R*np.exp(complex(0,1)* apertureP)
#V = [W/2,0; W/2,-W/2; -W/2,-W/2; -W/2,W/2; W/2,W/2; W/2,0; real(V),imag(V)];
#fdtd.set("vertices", "[1.0e-6, 2.0e-6; 1.5e-6,2.0e-6]")
#  setnamed("aperture","vertices",V);

# #==================================================
# Add the source 
# #==================================================
fdtd.addplane()
fdtd.set("name", "source")
fdtd.set("injection axis","z")
fdtd.set("direction","forward")
fdtd.set("x",0)
fdtd.set("x span",50.0e-6)

fdtd.set("y",0)
fdtd.set("y span",50.0e-6)

fdtd.set("z",-0.15e-6)

fdtd.set("center wavelength", 1.55e-6)
fdtd.set("wavelength span",0.0e-6)

# #==================================================
# Add the monitor
# #==================================================
fdtd.addpower()
fdtd.set("name","field")
fdtd.set("monitor type",7)  # 2D z-normal
fdtd.set("x",0)
fdtd.set("x span",25.0e-6)
fdtd.set("y",0)
fdtd.set("y span",25.0e-6)
fdtd.set("z",1.365e-6)
# #==================================================

fdtd.setglobalmonitor("frequency points", 1) # setting the global frequency resolution
# set("record electrostatic potential",1);
# set("save data",1);
# filename = "electric_field.mat";
# set("filename",filename);

# fdtd.addanalysisgroup()
# fdtd.set("name","model")
# fdtd.addtogroup("metalens")


# M=readdata("..//output_x_y_rad_apexPHAZE_optimized.txt");
# len= length(M)/3;
# for(i=1:len) {
#     addcircle({"name":"pillar", "x":M(i),"y":M(len+i),"radius":M(2*len+i)});
    
# }

#)











# ## Step 4: Create the structures

# addrect; # substrate
# set("name","substrate");
# set("material", "sio2");
# if (mat_sub=="<Object defined dielectric>") {
#     set("index",index_sub);
# }
# set("x span",3*max(x_mask));
# set("y span",3*max(x_mask));
# set("z max",0);
# set("z min",-2*wavelength);


# M=readdata("..//output_x_y_rad_apexPHAZE_optimized.txt");
# len= length(M)/3;
# for(i=1:len) {
#     addcircle({"name":"pillar", "x":M(i),"y":M(len+i),"radius":M(2*len+i)});
    
# }


# select("pillar");
# set("index",index_pillar);
# set("material", "si");


# set("z min",0);
# set("z max",1.3e-6);
# ?height;
# set("detail",min([0,%pillar rendering detail%]));
# fdtd.adduserprop("radius",2,0.5e-6)

# myscript =      "addcircle; \n"
# myscript = myscript + "copy(1e-6); \n"
# myscript = myscript + "selectall; \n"
# myscript = myscript + "set(\"radius\",radius);"
# fdtd.set("name","dimer")
# fdtd.set("script",myscript)



In [64]:
# apertureP= np.linspace(0, 2*np.pi, 200)
# V = R*np.exp(complex(0,1)* apertureP)
# V = [W/2,0; W/2,-W/2; -W/2,-W/2; -W/2,W/2; W/2,W/2; W/2,0; real(V),imag(V)];

[ 1.00000000e-05  9.99501589e-06  9.98006853e-06  9.95517283e-06
  9.92035359e-06  9.87564552e-06  9.82109320e-06  9.75675100e-06
  9.68268305e-06  9.59896320e-06  9.50567489e-06  9.40291112e-06
  9.29077433e-06  9.16937628e-06  9.03883800e-06  8.89928962e-06
  8.75087022e-06  8.59372777e-06  8.42801891e-06  8.25390881e-06
  8.07157104e-06  7.88118735e-06  7.68294752e-06  7.47704916e-06
  7.26369751e-06  7.04310526e-06  6.81549228e-06  6.58108547e-06
  6.34011850e-06  6.09283155e-06  5.83947114e-06  5.58028982e-06
  5.31554594e-06  5.04550341e-06  4.77043141e-06  4.49060414e-06
  4.20630055e-06  3.91780402e-06  3.62540213e-06  3.32938637e-06
  3.03005180e-06  2.72769681e-06  2.42262280e-06  2.11513386e-06
  1.80553651e-06  1.49413936e-06  1.18125282e-06  8.67188782e-07
  5.52260311e-07  2.36781335e-07 -7.89336691e-08 -3.94569991e-07
 -7.09812996e-07 -1.02434845e-06 -1.33786280e-06 -1.65004355e-06
 -1.96057949e-06 -2.26916109e-06 -2.57548074e-06 -2.87923309e-06
 -3.18011536e-06 -3.47782